# A little extra!

## New addition to Week 1

### The Unreasonable Effectiveness of the Agent Loop

# What is an Agent?

## Three competing definitions

1. AI systems that can do work for you independently - Sam Altman

2. A system in which an LLM controls the workflow - Anthropic

3. An LLM agent runs tools in a loop to achieve a goal

## The third one is the new, emerging definition

But what does it mean?

Let's make it real.

In [5]:
# Start with some imports - rich is a library for making formatted text output in the terminal

from rich.console import Console
from dotenv import load_dotenv
import os
from openai import OpenAI
import json
load_dotenv(override=True)

True

In [6]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [7]:
client = OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

In [8]:
# Some lists!

todos = []
completed = []

In [9]:
def get_todo_report() -> str:
    result = ""
    for index, todo in enumerate(todos):
        if completed[index]:
            result += f"Todo #{index + 1}: [green][strike]{todo}[/strike][/green]\n"
        else:
            result += f"Todo #{index + 1}: {todo}\n"
    show(result)
    return result

In [10]:
get_todo_report()

''

In [11]:
def create_todos(descriptions: list[str]) -> str:
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report()

In [12]:
def mark_complete(index: int, completion_notes: str) -> str:
    if 1 <= index <= len(todos):
        completed[index - 1] = True
    else:
        return "No todo at this index."
    Console().print(completion_notes)
    return get_todo_report()

In [13]:
todos, completed = [], []

create_todos(["Buy groceries", "Finish extra lab", "Eat banana"])

Todo #1: Buy groceries
Todo #2: Finish extra lab
Todo #3: Eat banana

'Todo #1: Buy groceries\nTodo #2: Finish extra lab\nTodo #3: Eat banana\n'

In [14]:
mark_complete(1, "bought")

bought

Todo #1: Buy groceries
Todo #2: Finish extra lab
Todo #3: Eat banana

'Todo #1: [green][strike]Buy groceries[/strike][/green]\nTodo #2: Finish extra lab\nTodo #3: Eat banana\n'

In [15]:
create_todos_json = {
    "name": "create_todos",
    "description": "Add new todos from a list of descriptions and return the full list",
    "parameters": {
        "type": "object",
        "properties": {
            "descriptions": {
                'type': 'array',
                'items': {'type': 'string'},
                'title': 'Descriptions'
                }
            },
        "required": ["descriptions"],
        "additionalProperties": False
    }
}

In [16]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Mark complete the todo at the given position (starting from 1) and return the full list",
    "parameters": {
        'properties': {
            'index': {
                'description': 'The 1-based index of the todo to mark as complete',
                'title': 'Index',
                'type': 'integer'
                },
            'completion_notes': {
                'description': 'Notes about how you completed the todo in rich console markup',
                'title': 'Completion Notes',
                'type': 'string'
                }
            },
        'required': ['index', 'completion_notes'],
        'type': 'object',
        'additionalProperties': False
    }
}

In [17]:
tools = [{"type": "function", "function": create_todos_json},
        {"type": "function", "function": mark_complete_json}]

In [18]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [19]:
def loop(messages):
    done = False
    while not done:
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",  # or llama-3.3-70b-versatile
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )

        finish_reason = response.choices[0].finish_reason

        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls

            results = handle_tool_calls(tool_calls)

            messages.append({
                "role": message.role,
                "content": message.content,
                "tool_calls": message.tool_calls
            })

            messages.extend(results)
        else:
            done = True

    print(response.choices[0].message.content)

In [20]:
system_message = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
If any quantity isn't provided in the question, then include a step to come up with a reasonable estimate.
Provide your solution in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """"
A train leaves Boston at 2:00 pm traveling 60 mph.
Another train leaves New York at 3:00 pm traveling 80 mph toward Boston.
When do they meet?
"""
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

In [21]:
todos, completed = [], []
loop(messages)

Todo #1: Estimate the distance from Boston to New York
Todo #2: Work out how far Boston train travels before second train starts
Todo #3: Calculate how soon after the second train starts do they meet
Todo #4: Write the solution for when they meet

Todo #1: Estimate the distance from Boston to New York
Todo #2: Work out how far Boston train travels before second train starts
Todo #3: Calculate how soon after the second train starts do they meet
Todo #4: Write the solution for when they meet
Todo #5: Assume an hourly speed of 60 mph from Boston and 80 mph from New York
Todo #6: Convert 60 and 80mph into miles per hour in the US (as mph)
Todo #7: Use 80mph as the relative speed between the two trains once they meet

Todo #1: Estimate the distance from Boston to New York
Todo #2: Work out how far Boston train travels before second train starts
Todo #3: Calculate how soon after the second train starts do they meet
Todo #4: Write the solution for when they meet
Todo #5: Assume an hourly speed of 60 mph from Boston and 80 mph from New York
Todo #6: Convert 60 and 80mph into miles per hour in the US (as mph)
Todo #7: Use 80mph as the relative speed between the two trains once they meet
Todo #8: Work out how long it takes the two trains to meet once they start moving towards each other
Todo #9: Work out the exact time they will meet once one train is already travelling for an hour
Todo #10: Find the exact time when the two trains meet

The two trains meet at 5:20 pm

Todo #1: Estimate the distance from Boston to New York
Todo #2: Work out how far Boston train travels before second train starts
Todo #3: Calculate how soon after the second train starts do they meet
Todo #4: Write the solution for when they meet
Todo #5: Assume an hourly speed of 60 mph from Boston and 80 mph from New York
Todo #6: Convert 60 and 80mph into miles per hour in the US (as mph)
Todo #7: Use 80mph as the relative speed between the two trains once they meet
Todo #8: Work out how long it takes the two trains to meet once they start moving towards each other
Todo #9: Work out the exact time they will meet once one train is already travelling for an hour
Todo #10: Find the exact time when the two trains meet

The two trains meet at 5:20 pm.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Now try to build an Agent Loop from scratch yourself!<br/>
            Create a new .ipynb and make one from first principles, referring back to this as needed.<br/>
            It's one of the few times that I recommend typing from scratch - it's a very satisfying result.
            </span>
        </td>
    </tr>
</table>